## Recognize Text from Unstructured Scene
- [original discussion](http://www.mathworks.com/help/vision/examples/automatically-detect-and-recognize-text-in-natural-images.html)
- Approach
    - Use [MSER](http://stackoverflow.com/questions/17647500/exact-meaning-of-the-parameters-given-to-initialize-mser-in-opencv-2-4-x) to detect potential regions containing text
    - process and merge the region, make it in appropriate size
    - use OCR (tesseract) to recognize the text

In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from skimage import io, exposure, measure, morphology
import cv2